# 3D Brain Visualizations

## 3D topographic brain map

The 3D topographic brain map provides a view of voltage measurements as a heatmap converted to estimated position on the brain. There are 2 plot options with different backends: 

1) [matplotlib](#matplotlib) (the version found on the UI)

2) [pyVista](#pyvista) (a better rendering but incompatible with the UI)

Both plots can be generated as an animation to view changes over time or as a standalone plot.

![](instruction_imgs/presentattion_brain.gif)

## General Setup
### Import required modules

In [ ]:
from simpl_eeg import topomap_3d_brain, eeg_objects

In [ ]:
import warnings
warnings.filterwarnings('ignore')

```{note}
Please include the line below in your IDE so that the changes would be simultaneously reflected when you make a change to the python scripts.
```

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

### Create epoched data
For additional options see [Creating EEG Objects](eeg_objects.html#intro) section.

In [ ]:
experiment_folder = "../../data/109"
epochs = eeg_objects.Epochs(experiment_folder)

frame_steps = 100
epoch = epochs.skip_n_steps(frame_steps)

### Generate forward and inverse (optional)

```{note}
- Before an animation or plot can be generated, an **"inverse"** (abbreviated as **"stc"**) must first be generated. An inverse can be generated from an epoch and a **"forward"** (abbreviated as **"fwd"**), which itself can be generated from an epoch alone. If only an epoch is provided to either of the plotting functions the stc and fwd will be automatically generated, **HOWEVER** this will increase the time it takes to generate the figure.

- The forward/inverse are used to retrieve a brain model to attach the EEG data and to do some of the mapping calculations. The forward downloads 'fsaverage' MRI data which represents a brain averaged out from dozens of different patients.

- You may pass one of three combinations to generate a figure in either of the plotting functions.
    1. An 'stc' (fastest)
    2. An 'epoch' AND a 'fwd' (fast)
    3. An 'epoch' (slow)
```

#### Generate Forward

In [ ]:
fwd = topomap_3d_brain.create_fsaverage_forward(epoch)

#### Generate Inverse

In [ ]:
stc = topomap_3d_brain.create_inverse_solution(epoch, fwd)

<a id="matplotlib"></a>
## Create a matplotlib 3D brain animation

### Simple Plot (+ auto-generating forward and inverse)

#### Define parameters (simple)

A detailed description of all parameters can be found in the `topomap_3d_brain.animate_matplot_brain` docstring:

In [ ]:
help(topomap_3d_brain.animate_matplot_brain)

#### Define parameters

In [ ]:
# change values below to values of interest

# arguments built into the package
stc = stc
views=['lat', 'dor']
size=200
hemi='both'
colormap='mne'
colorbar=True
colormap_limit_type='lims'
cmin=-2
cmax=2
spacing='oct5'
smoothing_steps=5
timestamp=True
frame_rate=12


# some useful (and tested) arguments from the
# MNE.viz.plot_topomap function (see **kwargs) include...
surface='inflated'
cortex='classic'

```{note}
Remember there are three options for passing a combination of epoch, fwd, and stc to generate a plot. These are...
1.
    stc=stc

2.
    epoch=epoch,
    fwd=fwd

3.
    epoch=epoch
```

#### Generate animation with matplotlib backend

In [ ]:
%%capture

matplotlib_animation = topomap_3d_brain.animate_matplot_brain(
    stc=stc,
    views=views,
    hemi=hemi,
    colormap=colormap,
    colorbar=colorbar,
    colormap_limit_type=colormap_limit_type,
    cmin=cmin,
    cmax=cmax,
    spacing=spacing,
    smoothing_steps=smoothing_steps,
    timestamp=timestamp,
    frame_rate=frame_rate,
    surface=surface,
    cortex=cortex
)

from IPython.display import HTML
video = HTML(matplotlib_animation.to_jshtml())

In [ ]:
video

### Saving the animation

#### Save as gif

```python
anim_brain = topomap_3d_brain.animate_matplot_brain(stc = stc, views = 'lat', hemi = 'lh')

gif_file_path = "examples/topomap_3d_brain.gif" 
anim_brain.save(gif_file_path, fps=5, dpi=300)
```

#### Save as mp4

```python
mp4_file_path = "examples/topo_2d.mp4"
anim_brain.save(mp4_file_path, fps=5, dpi=300)
```

```{note}
If `FFMpegWriter` does not work on your computer you can save the file as a gif first and then convert it into mp4 file by running the code below.
```
```python
import moviepy.editor as mp

clip = mp.VideoFileClip(gif_file_path)
clip.write_videofile(mp4_file_path)
```

## Create a matplotlib 3D brain figure

### Generating a matplotlib plot

### Define parameters
A detailed description of all animation parameters can be found in the `topomap_3d_brain.plot_topomap_3d_brain` docstring:

In [ ]:
help(topomap_3d_brain.plot_topomap_3d_brain)

In [ ]:
%%capture
matplot_brain_fig = topomap_3d_brain.plot_topomap_3d_brain(stc=stc,
                                                           recording_number=2,
                                                           hemi='lh',
                                                           views=['lat'],
                                                           cmin=-2,
                                                           cmax=2)

In [ ]:
matplot_brain_fig

### Save the plot
You can change the file to different formats by changing the format argument in the function. It supports `png`, `pdf`, `svg`.
```python
file_path = "examples/topomap_3d_brain.svg"  
matplot_brain_fig.savefig(file_path, format='svg')
```

<a id="pyvista"></a>
## Create a pyVista 3D brain animation

```{note}
The dependencies for the following functions are not included with the base instalation of the simpl_eeg package due to compatability issues with certain operating systems. If you wish to use them please install 'PyVista', 'PyQt5', 'pyvistaqt', and 'pyqt5-qt5' through your preferred package installation method https://mne.tools/stable/install/mne_python.html.
```

### Generate figure with pyvista backend

```python
pyvista_brain_fig = topomap_3d_brain.plot_topomap_3d_brain(stc = stc, backend = 'pyvista')
```

### Save animation with pyvista backend

```python
topomap_3d_brain.save_animated_topomap_3d_brain(pyvista_brain_fig, filename = "brain_animation.gif")
```